Employment trends by race and disability
==================================

How does *prime-age employment* (the employment rate of civilians aged 25 to 54)
vary with race and disability status? How does disability status vary by race?
How do race and disability status compare in predicting employment?

In this post, I use the Current Population Survey to find out.
Overall, I find that the disability employment gap is large---41 percentage points on average in May 2020.
The racial employment gap (non-Black minus Black) is 7 points, but since Black people are about 30 percent more likely
to have a disability, this gap shrinks to 6.3 points when controlling for disability status.
Among people with disabilities, black people have employment rates 11 points lower than
non-black people (a 32 percent difference).
I also find that the coronavirus recession widened the racial employment gap, primarily
among the population without disabilities.

## Background

The prime-age employment rate (often abbreviated as *PAEPOP*, for Prime Age Employment POPulation ratio)
is used as a consistent measure of employment trends by economists who
want to avoid relying on whether survey respondents say they're still looking for work.
The calculation is simply the share of civilians aged 25 to 54 that reported working in the survey week.

[Federal Reserve Economic Data](http://fred.stlouisfed.org/) publishes some related trends,
e.g. [PAEPOP by disability and gender](https://fred.stlouisfed.org/series/LNU02376960)
and [black employment rate among the aged 16+ population](https://fred.stlouisfed.org/series/LNS12300006),
but we need to limit by ages to avoid effects of college education and early retirement,
both of which may reflect lower employment without worse labor market outcomes.
For the right statistics, we need to go to the CPS microdata instead (I used [IPUMS](http://ipums.org) to extract it).

For simplicity, I focus on two binary features identified in the CPS:
being Black only (compared to all other races), and
reporting [any physical or cognitive difficulty](https://cps.ipums.org/cps-action/variables/DIFFANY#description_section) (which I describe here as having a disability).

## Results

From 2017 until the coronavirus hit, the employment gap between Black people and non-Black people had been roughly steady at around 4 percentage points.
But the pandemic hurt Black people disproportionately: in April, the gap grew to 6 points, 
and even as both groups recovered in May, it grew further to 7 points.

In [151]:
### LOAD PACKAGES ###

import pandas as pd
import numpy as np
import microdf as mdf
import plotly.express as px
import statsmodels.api as sm
import stargazer.stargazer as sg


### SETTINGS ###

# Colors from https://material.io/design/color/the-color-system.html
BLUE = '#1976D2'
GRAY = '#BDBDBD'
RED = '#C62828'
PURPLE = '#6A1B9A'
LIGHT_BLUE = '#64B5F6'
PINK = '#EF9A9A'

COLOR_MAP = {
    'Black': BLUE,
    'Not Black': GRAY,
    'Has disability': RED,
    'No disability': GRAY,
    'Black, Has disability': PURPLE,
    'Black, No disability': LIGHT_BLUE,
    'Not Black, Has disability': PINK,
    'Not Black, No disability': GRAY
}


### LOAD DATA ###

# CPS data extracted from IPUMS.
cps_raw = pd.read_csv('cps.csv.gz')

### PREPROCESS ###

cps = cps_raw.copy(deep=True)
cps.columns = cps.columns.str.lower()
cps.rename({'wtfinl': 'w'}, axis=1, inplace=True)
cps['day'] = 12  # CPS asks about week including 12th of the month.
cps['date'] = pd.to_datetime(cps[['year', 'month', 'day']])
# Create descriptive bools from codes.
cps['female'] = cps.sex == 2
cps['emp'] = cps.empstat.isin([10, 12])
cps['black'] = np.where(cps.race == 200, 'Black', 'Not Black')
# Recode disability into True/False/None.
DIFFS = ['diffhear', 'diffeye', 'diffrem', 'diffphys', 'diffmob', 'diffcare']
for i in DIFFS + ['diffany']:
    cps[i] = np.where(cps[i] == 0, np.nan, cps[i] == 2)
    
# Limit to prime-age civilians and relevant columns.
# COLS = ['emp', 'date', 'w', 'black', 'diffany']
cps = cps[(cps.empstat != 1)  # Not in armed forces.
          & cps.age.between(25, 54)]
assert cps.diffany.isna().sum() == 0
cps['disability'] = np.where(cps.diffany == 1, 'Has disability',
                             'No disability')

### ANALYSIS ###

# Create grouped dataframe.
grouped = cps.groupby(['date', 'black', 'disability', 'emp'])[['w']].sum()
grouped.reset_index(inplace=True)
# Add conditional columns for creating rates.
mdf.add_weighted_metrics(grouped, ['emp'], 'w')
grouped['disability_m'] = np.where(grouped.disability == 'Has disability',
                                   grouped.w_m, 0)

def add_emp_rate(df):
    df['emp_rate'] = 100 * df.emp_m / df.w_m
    df['emp_rate_round'] = df.emp_rate.round(1)
    df.drop(['emp_m', 'w_m'], axis=1, inplace=True)
    df.reset_index(inplace=True)
    
def add_disability_rate(df):
    df['disability_rate'] = 100 * df.disability_m / df.w_m
    df['disability_rate_round'] = df.disability_rate.round(1)
    df.drop(['disability_m', 'w_m'], axis=1, inplace=True)
    df.reset_index(inplace=True)
    
race_emp = grouped.groupby(['date', 'black'])[['emp_m', 'w_m']].sum()
add_emp_rate(race_emp)

disability_emp = grouped.groupby(['date', 'disability'])[
    ['emp_m', 'w_m']].sum()
add_emp_rate(disability_emp)

race_disability = grouped.groupby(['date', 'black'])[
    ['disability_m', 'w_m']].sum()
add_disability_rate(race_disability)

race_disability_emp = grouped.groupby(['date', 'black', 'disability'])[
    ['emp_m', 'w_m']].sum()
add_emp_rate(race_disability_emp)
race_disability_emp['label'] = (race_disability_emp.black + ', ' +
                                race_disability_emp.disability)

def line_graph(df, x, y, color, title, yaxis_title):
    fig = px.line(df, x=x, y=y, color=color, color_discrete_map=COLOR_MAP)
    fig.update_layout(
        title=title,
        xaxis_title='',
        yaxis_title=yaxis_title,
        yaxis_ticksuffix='%',
        legend_title_text='',
        font=dict(family='Roboto'),
        hovermode='x',
        plot_bgcolor='white'
    )
    fig.update_traces(mode='markers+lines', hovertemplate=None)

    fig.show()

line_graph(df=race_emp, x='date', y='emp_rate_round', color='black',
           title='Prime-age employment rate by race',
           yaxis_title='Employment rate of civilians aged 25 to 54')

Black people are also about 30 percent more likely to have a disability than non-Black people, with rates of about 7.5 percent compared to 5.7 percent, respectively. Small samples make this a noisy signal on a month-to-month basis: only 301 of the 36,000 prime-age civilian respondents in the April 2020 survey reported being Black and having a disability.

In [152]:
line_graph(race_disability, x='date', y='disability_rate_round', color='black',
           title='Disability rate by race, civilians aged 25 to 54',
           yaxis_title=
           'Share of civilians aged 25 to 54 who report any difficulty')

The disability employment gap is very large, but it has shrunk over time.
From January 2017 to January 2020, it fell from 50 percentage points
(30 percent among people with disabilities and 80 percent among people without disabilities)
to 46 points (37 percent and 83 percent).
Coronavirus has actually shrunk the gap further: since January, employment of people with disabilities
has fallen 5 points, compared to 9 points among people without disabilities.

In [153]:
line_graph(disability_emp, x='date', y='emp_rate_round', color='disability',
           title='Prime-age employment rate by disability status',
           yaxis_title='Employment rate of civilians aged 25 to 54')

Breaking out the trends by both race and disability status reveals that
the racial employment gap among people without disabilities has been steady around 3 percentage points since 2017, but
has roughly doubled since coronavirus.
The racial employment gap among people with disabilities has been noisy at around 10 points,
and does not appear to have changed significantly as a result of coronavirus.

In [154]:
line_graph(race_disability_emp, x='date', y='emp_rate_round', color='label',
           title='Prime-age employment rate by race and disability status',
           yaxis_title='Employment rate of civilians aged 25 to 54')

Honing in on the latest month of data (May 2020) emphasizes the 50-percent-larger racial employment gap
among people with disabilities.
Put another way: among people without disabilities, Black people are 8 percent less likely to be
employed than non-Black people, while they're 32 percent less likely to be employed among people *with* disabilities.

In [155]:
race_disability_emp_latest = race_disability_emp[
    race_disability_emp.date == race_disability_emp.date.max()].copy(deep=True)
race_disability_emp_latest.sort_values('black', ascending=False, inplace=True)

fig = px.bar(race_disability_emp_latest, x='disability', y='emp_rate_round',
             color='black', barmode='group',
             color_discrete_map=COLOR_MAP)
fig.update_layout(
    title='Prime-age employment rate by race and disability status,' +
    ' May 2020',
    xaxis_title='',
    yaxis_title='Employment rate of civilians aged 25 to 54',
    yaxis_ticksuffix='%',
    legend_title_text='',
    plot_bgcolor='white',
    font=dict(family='Roboto'),
    xaxis={'categoryorder':'total descending'}
)
fig.update_traces(hovertemplate=None)
fig.show()

How to tease out the effects of being Black vs. having a disability, when they interact and each 
intersectional subgroup differs in size?
Regression, of course!

A linear probability regression shows that, in May 2020, being Black had an average employment effect of -6.3 percentage points,
while having a disability had an average effect of -41.0 percentage points (about 6.5x the Black effect).

In [156]:
cps.columns

Index(['year', 'month', 'w', 'age', 'sex', 'race', 'empstat', 'diffhear',
       'diffeye', 'diffrem', 'diffphys', 'diffmob', 'diffcare', 'diffany',
       'day', 'date', 'female', 'emp', 'black', 'disability'],
      dtype='object')

In [157]:
cps_ly = cps[cps.date > '2019-05-30'].copy(deep=True)
# cps2020 = cps[cps.date.dt.year == 2020].copy(deep=True)
cps_ly['covid'] = cps_ly.date.dt.month > 3

# Switch text columns to bools (and later ints).
cps_ly['black'] = cps_ly.black == 'Black'
cps_ly['disability'] = cps_ly.disability == 'Has disability'
# Create interactions.
cps_ly['black_disability'] = cps_ly.black & cps_ly.disability
cps_ly['black_covid'] = cps_ly.black & cps_ly.covid
cps_ly['disability_covid'] = cps_ly.disability & cps_ly.covid
cps_ly['black_disability_covid'] = cps_ly.black_disability & cps_ly.covid

# Add fixed effects for each month.
for i in cps_ly.date.dt.month.unique():
    cps_ly['month' + str(i)] = cps_ly.date.dt.month == i

# No longer need date.
cps_ly.drop('date', axis=1, inplace=True)

# Add other controls.
cps_ly['age2'] = np.power(cps_ly.age, 2)

# Add a constant and change to ints
cps_ly['const'] = 1
cps_ly *= 1

DD_COLS = ['black', 'disability', 'black_disability']
MONTH_FES = ['month' + str(i) for i in np.arange(1, 6)]
CONTROLS = ['age', 'age2', 'female']

def emp_model(xcols, data=cps_ly):
    return sm.WLS(data.emp,
                  data[xcols + ['const']],
                  data.w).fit(cov_type='HC1')


# Run model without triple-diff first.
m = emp_model(DD_COLS + MONTH_FES)

m_controls = emp_model(DD_COLS + MONTH_FES + CONTROLS)

m_controls_diffs = emp_model(DD_COLS + MONTH_FES + CONTROLS + DIFFS)

COVARIATE_NAMES = {
    'black': 'Black',
    'disability': 'Has disability',
    'black_disability': 'Black * Has disability',
    # Name difficulties following IPUMS labels.
    'diffhear': 'Hearing difficulty',
    'diffeye': 'Vision difficulty',
    'diffrem': 'Difficulty remembering',
    'diffphys': 'Physical difficulty', 
    'diffmob': 'Mobility limitation',
    'diffcare': 'Personal care difficulty',
    'diffhear_black': 'Hearing difficulty * Black',
    'diffeye_black': 'Vision difficulty * Black',
    'diffrem_black': 'Difficulty remembering * Black',
    'diffphys_black': 'Physical difficulty * Black', 
    'diffmob_black': 'Mobility limitation * Black',
    'diffcare_black': 'Personal care difficulty * Black',
}

def starg(models, covariate_order):
    """ Creates formatted Stargazer object.
    """
    star = sg.Stargazer(models)
    star.covariate_order(covariate_order)
    star.rename_covariates(COVARIATE_NAMES)
    # These aren't currently merged, see
    # https://github.com/mwburke/stargazer/pull/58.
    star.hide_adj_r2()
    star.hide_residual_std_err()
    star.hide_f_statistic()
    return star


starg([m, m_controls, m_controls_diffs],
      ['black', 'disability', 'black_disability'])

Which disabilities are Black people more likely to have?

Black Americans are [30 percent more likely](https://minorityhealth.hhs.gov/omh/browse.aspx?lvl=4&lvlid=25) to be obese.

In [158]:
DIFFS

['diffhear', 'diffeye', 'diffrem', 'diffphys', 'diffmob', 'diffcare']

In [159]:
cps_ly['num_diffs'] = cps_ly[DIFFS].sum(axis=1)
mdf.add_weighted_metrics(cps_ly, 'num_diffs', 'w')
# Needs to be weighted.
num_diffs = cps_ly[cps_ly.disability == 1].groupby('black')[
    ['num_diffs_m', 'w_m']].sum()
num_diffs['num_diffs'] = num_diffs.num_diffs_m / num_diffs.w_m
num_diffs.num_diffs

black
0    1.698733
1    1.829746
Name: num_diffs, dtype: float64

In [160]:
def share_w_disability(disability, black):
    num = cps_ly[(cps_ly.black == black) & (cps_ly[disability] == 1)].w.sum()
    denom = cps_ly[(cps_ly.black == black) & (cps_ly.disability)].w.sum()
    return num / denom

diffs_black = mdf.cartesian_product({'disability': DIFFS,
                                     'black': [True, False]})
diffs_black['share'] = diffs_black.apply(
    lambda row: share_w_disability(row.disability, row.black), axis=1)
diffs_black.share *= 100
diffs_black['share_round'] = diffs_black.share.round(1)
diffs_black.black = np.where(diffs_black.black, 'Black', 'Not Black')
diffs_black.replace(DIFFS,
                    ['Hearing', 'Vision', 'Remembering', 'Physical',
                     'Mobility', 'Self-care'],
                    inplace=True)
# Sort by black for within-group ordering.
diffs_black.sort_values('black', ascending=False, inplace=True)

In [161]:
# Order chart by prevalence among Black people.
diff_order = diffs_black[diffs_black.black == 'Black'].sort_values(
    'share').disability

fig = px.bar(diffs_black, y='disability', x='share_round', color='black',
             barmode='group', orientation='h',
             color_discrete_map=COLOR_MAP)
fig.update_layout(
    title='Prevalence of each CPS difficulty by race, June 2019 to May 2020',
    yaxis_title='Difficulty',
    xaxis_title='Share among people who report any difficulty',
    xaxis_ticksuffix='%',
    legend_title_text='',
    font=dict(family='Roboto'),
    yaxis={'categoryorder': 'array',
           'categoryarray': diff_order},
    plot_bgcolor='white',
    legend={'traceorder': 'reversed'}
)
fig.update_traces(hovertemplate=None)
fig.show()

In [162]:
diff_order

0         Hearing
2          Vision
10      Self-care
8        Mobility
4     Remembering
6        Physical
Name: disability, dtype: object

How do the effects of each disability compare between Black and non-Black people?

In [163]:
cps_ly

,year,month,w,age,sex,race,empstat,diffhear,diffeye,diffrem,...,month1,month2,month3,month4,month5,age2,const,num_diffs,w_m,num_diffs_m
3606573,2019,6,2356.8863,25,1,100,10,0.0,0.0,0.0,...,0,0,0,0,0,625,1,0.0,0.002357,0.000000
3606574,2019,6,1815.7853,51,2,100,10,0.0,0.0,0.0,...,0,0,0,0,0,2601,1,0.0,0.001816,0.000000
3606575,2019,6,1823.4531,51,1,100,10,0.0,0.0,0.0,...,0,0,0,0,0,2601,1,0.0,0.001823,0.000000
3606578,2019,6,2439.2458,49,1,200,10,0.0,0.0,0.0,...,0,0,0,0,0,2401,1,0.0,0.002439,0.000000
3606585,2019,6,2342.9010,31,1,200,21,0.0,0.0,0.0,...,0,0,0,0,0,961,1,0.0,0.002343,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4973768,2020,5,348.9306,45,2,100,10,0.0,0.0,0.0,...,0,0,0,0,1,2025,1,0.0,0.000349,0.000000
4973772,2020,5,433.3859,41,1,100,10,0.0,0.0,0.0,...,0,0,0,0,1,1681,1,0.0,0.000433,0.000000
4973773,2020,5,400.0569,40,2,100,10,0.0,0.0,0.0,...,0,0,0,0,1,1600,1,0.0,0.000400,0.000000
4973780,2020,5,454.0825,41,2,100,10,0.0,0.0,0.0,...,0,0,0,0,1,1681,1,0.0,0.000454,0.000000


In [164]:
for i in DIFFS:
    cps_ly[i + '_black'] = cps_ly[i] * cps_ly.black
    
m_diffs = emp_model(['black'] + MONTH_FES + CONTROLS + DIFFS)
black_diffs = [i + '_black' for i in DIFFS]
m_diffs_black = emp_model(['black'] + MONTH_FES + CONTROLS + DIFFS + 
                          black_diffs)

starg([m_diffs, m_diffs_black],
      ['black'] + list(mdf.flatten([[i] + [i + '_black'] for i in DIFFS])))

In [165]:
list(mdf.flatten([[i] + [i + '_black'] for i in DIFFS]))

['diffhear',
 'diffhear_black',
 'diffeye',
 'diffeye_black',
 'diffrem',
 'diffrem_black',
 'diffphys',
 'diffphys_black',
 'diffmob',
 'diffmob_black',
 'diffcare',
 'diffcare_black']

In [166]:
cps_latest = cps[cps.date == cps.date.max()].copy(deep=True)
cps_latest.drop('date', axis=1, inplace=True)

cps_latest['is_black'] = cps_latest.black == 'Black'
cps_latest['has_disability'] = cps_latest.disability == 'Has disability'
cps_latest['is_black_has_disability'] = (
    cps_latest.is_black & cps_latest.has_disability)
cps_latest.drop(['black', 'disability'], axis=1, inplace=True)

cps_latest = sm.add_constant(cps_latest) * 1

m = sm.WLS(cps_latest.emp,
           cps_latest[['is_black', 'has_disability',
                       'is_black_has_disability', 'const']],
           cps_latest.w).fit(cov_type='HC1')

star = sg.Stargazer([m])
star.covariate_order(['is_black', 'has_disability', 'is_black_has_disability'])
star.rename_covariates({'is_black': 'Black',
                        'has_disability': 'Has disability',
                        'is_black_has_disability': 'Black * Has disability'})
star

KeyError: "['const'] not in index"

While people with disabilities represent a small part of the overall racial employment gap,
they constitute a particularly acute element of it.
As we rethink our approach to work in the age of coronavirus,
we will have opportunities to close part of the enormous disability employment gap,
which will in turn close part of the racial employment gap.